# 

In [1]:

!pip install -U scikit-learn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import pandas as pd
import sklearn as sk



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


2023-08-07 23:46:46.343436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_logical_devices()

2023-08-07 23:46:55.868116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-07 23:46:56.194868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-07 23:46:56.195072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-07 23:46:56.222568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-07 23:46:56.222762: I tensorflow/compile

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
x = pd.read_csv("Data/liar.csv")

In [4]:
x.describe()

,label,comment
count,7939,7939
unique,2,7929
top,False,On changing the rules for filibusters on presi...
freq,4389,3


In [5]:
x['text'] = x['comment']

In [6]:
x.drop(columns=['comment'])

,label,text
0,False,Says the Annies List political group supports ...
1,True,"Hillary Clinton agrees with John McCain ""by vo..."
2,False,Health care reform legislation is likely to ma...
3,True,The Chicago Bears have had more starting quart...
4,False,Jim Dunnam has not lived in the district he re...
...,...,...
7934,False,Says Rick Scott has teamed up with a felon con...
7935,True,The Castro regime violated international norms...
7936,True,Theres not one Democrat who endorses the Romne...
7937,True,The United States death rate is two-and-a-half...


In [7]:
x_FALSE = x[x['label'] == False]
x_TRUE = x[x['label'] == True]
x_FALSE.shape

(4389, 3)

In [11]:
x_TRUE.shape

(3550, 3)

In [14]:
x_FALSE = x_FALSE[:len(x_TRUE)]
x_FALSE.shape

(3550, 3)

In [15]:
x_balanced = pd.concat([x_TRUE, x_FALSE], axis=0)

In [16]:
x_balanced.dtypes

label        bool
comment    object
text       object
dtype: object

In [17]:
x_balanced.sample(10)

,label,comment,text
567,False,"Says Mahatma Gandhi once said, First they igno...","Says Mahatma Gandhi once said, First they igno..."
4517,True,Says Bernie Sanders chose to spend his honeymo...,Says Bernie Sanders chose to spend his honeymo...
4638,True,Says Alex Jones said that the victims of the S...,Says Alex Jones said that the victims of the S...
105,True,You had more women getting computer sciences d...,You had more women getting computer sciences d...
3949,False,Gonzales losing campaign against me last sprin...,Gonzales losing campaign against me last sprin...
3635,True,Almost 10 percent of Canadians came to visit F...,Almost 10 percent of Canadians came to visit F...
4796,False,Birth control pioneer Margaret Sanger believed...,Birth control pioneer Margaret Sanger believed...
7397,True,Property taxes had increased 70 percent in the...,Property taxes had increased 70 percent in the...
4828,True,African-Americans dont use drugs at a higher l...,African-Americans dont use drugs at a higher l...
5785,False,We have been confronted with an unexpected sto...,We have been confronted with an unexpected sto...


In [18]:
x_balanced.drop(columns=['comment'])

,label,text
1,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,True,The Chicago Bears have had more starting quart...
5,True,Says GOP primary opponents Glenn Grothman and ...
6,True,"For the first time in history, the share of th..."
8,True,The economy bled $24 billion due to the govern...
...,...,...
6456,False,Says Texas schools have had massive increases ...
6457,False,Paul Workman wants to raise the sales tax.
6458,False,Says Sen. Sherrod Brown cast the deciding vote...
6460,False,"I wanted to keep it private, because I dont th..."


In [19]:
x_balanced["text"] = x_balanced["text"].astype(str)

In [20]:
x_balanced['label'] = x_balanced['label'].apply(lambda x: 1 if x == False else 0)

In [21]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_balanced["text"], x_balanced["label"], test_size=0.2, random_state=42, stratify=x_balanced["label"])

In [22]:
#import tensorflow hub
!pip install tensorflow-hub
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text
import tensorflow_text as text


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [23]:
#grab the bert model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [24]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [26]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [27]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


2023-08-07 23:58:36.079224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-07 23:58:36.913167: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fae3802c430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 23:58:36.913209: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-08-07 23:58:37.446991: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-07 23:58:38.553270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


  1/178 [..............................] - ETA: 45:50 - loss: 0.7564 - accuracy: 0.4688 - precision: 0.5882 - recall: 0.5000

2023-08-07 23:58:39.701732: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


178/178 [==============================] - 56s 231ms/step - loss: 0.7361 - accuracy: 0.5192 - precision: 0.5192 - recall: 0.5183
Epoch 2/5
178/178 [==============================] - 41s 230ms/step - loss: 0.7175 - accuracy: 0.5241 - precision: 0.5243 - recall: 0.5208
Epoch 3/5
178/178 [==============================] - 41s 230ms/step - loss: 0.7152 - accuracy: 0.5349 - precision: 0.5354 - recall: 0.5271
Epoch 4/5
178/178 [==============================] - 41s 230ms/step - loss: 0.6986 - accuracy: 0.5548 - precision: 0.5547 - recall: 0.5549
Epoch 5/5
178/178 [==============================] - 41s 230ms/step - loss: 0.6967 - accuracy: 0.5570 - precision: 0.5570 - recall: 0.5577
